In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

In [4]:
df = pd.read_csv("data/train.csv")

In [5]:
class Attention(nn.Module):
    def __init__(self, feature_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(feature_dim, 1)

    def forward(self, x):
        attn_scores = self.attention_weights(x)  # Compute attention scores
        attn_scores = F.softmax(attn_scores, dim=1)  # Softmax to get attention weights
        weighted_x = x * attn_scores  # Apply attention weights
        return torch.sum(weighted_x, dim=1)  # Summing across features

class EEG_CNN_Attention(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EEG_CNN_Attention, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.attention = Attention(feature_dim=64)
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension for CNN
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.permute(0, 2, 1)  # Permute for attention mechanism
        x = self.attention(x)  # Apply attention
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Model initialization
num_features = 77 + 2  # 77 EEG features + age + IQ
num_classes = len(set(df['main.disorder']))  # Number of disorder categories

model = EEG_CNN_Attention(input_dim=num_features, num_classes=num_classes)
print(model)

EEG_CNN_Attention(
  (conv1): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (attention): Attention(
    (attention_weights): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
)


In [ ]:
# Define dataset class
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataloaders
train_dataset = EEGDataset(X_train_tensor, y_train_tensor)
val_dataset = EEGDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Load dataset
data = df.copy()

# Select relevant features and target
target_column = "main.disorder"
feature_columns = existing_columns + ["age", "iq"]
X = data[feature_columns]
y = data[target_column]

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)